# Ejercicio 12: Web Scraping
## Objetivo de la práctica
El objetivo de este ejercicio es construir un web scraper que recoja datos de un website.

## Parte 0: Planificar

1. Identificar los datos que quieres obtener.
2. Elegir el sitio web objetivo.
3. Planificar la estructura del corpus.

# Parte 1: Entender el sitio web objetivo

* Analizar la estructura de la página web a ser analizada.
* Identificar los elementos HTML que contienen los datos bsuscados.


In [1]:
# RAG
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.1 MB/s eta 0:00:00:00:0100:01


In [2]:
import openai
import faiss
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import Image, display
from sentence_transformers import SentenceTransformer
import os
import requests
import numpy as np

2025-07-27 20:31:40.209225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753648300.411417      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753648300.472029      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
#Open AI 

import os
from openai import OpenAI

#Obtenemos el secret_key
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("key") 

# Inicializamos el cliente de OpenAI
client = OpenAI(api_key=api_key)

# Creamos la petición al modelo GPT
response = client.responses.create(
    model="gpt-4o",
    #model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Under the silvery glow of the moon, Luna the unicorn softly trotted across the stardust-kissed meadows, weaving dreams of wonder for all the sleeping children.


In [4]:
file = "/kaggle/input/recetas/rotisserie-chicken.html"

#Cargamos el archivo html
with open(file, "r", encoding='utf-8') as file:
    html_content = file.read()

#Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

In [5]:
# Extracting the recipe title
title = soup.find("meta", {"property": "og:title"})["content"]
title

'Rotisserie Chicken'

In [6]:
#Ingredientes 
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
for ingredient in ingredients_section:
    print(ingredient.text.strip())

1 (3 pound) whole chicken
1 pinch salt
¼ cup butter, melted
1 tablespoon salt
1 tablespoon ground paprika
¼ tablespoon ground black pepper


# Parte 2: Obtener los datos deseados
* Buscar dentro del contenido HTML y extraer la información.

In [7]:
# Extracting the description
description = soup.find("meta", {"name": "description"})["content"]

# Extracting the ingredients
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
ingredients = [ingredient.get_text().strip() for ingredient in ingredients_section]

# Extracting the instructions
instructions_section = soup.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
instructions = [instruction.get_text().strip() for instruction in instructions_section]

# Extracting the nutrition information
nutrition_section = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")
nutrition_facts = [fact.parent.get_text().strip().replace('\n', ' ') for fact in nutrition_section]

# Print the extracted information
print("Recipe Title:", title)
print("Description:", description)
print("Ingredients:")
for ingredient in ingredients:
    print("-", ingredient)
print("Instructions:")
for i, instruction in enumerate(instructions, 1):
    print(f"{i}. {instruction}")
print("Nutrition Facts:")
for fact in nutrition_facts:
    print("-", fact)

Recipe Title: Rotisserie Chicken
Description: Rotisserie chicken that's easy to cook on a gas grill and turns out moist and juicy with crispy skin. This is a simple recipe that our family loves.
Ingredients:
- 1 (3 pound) whole chicken
- 1 pinch salt
- ¼ cup butter, melted
- 1 tablespoon salt
- 1 tablespoon ground paprika
- ¼ tablespoon ground black pepper
Instructions:
1. Intimidated by the idea of making a rotisserie chicken at home? We're here to help. Get your grill and rotisserie attachment ready — you'll want to try this recipe ASAP.
2. Here's what you'll need to make rotisserie chicken at home:
3. · Whole Chicken: This recipe is meant for a whole 3-pound chicken. If your chicken is larger or smaller, you'll have to adjust the cooking time.· Butter: Butter keeps the chicken moist and juicy, while giving the seasonings something to stick to.· Seasonings: The rotisserie chicken is simply seasoned with salt, pepper, and paprika.
4. You'll find the full, step-by-step recipe below — b

In [7]:
def get_details(soup):
    # Extracting the recipe title
    title = soup.find("meta", {"property": "og:title"})["content"].strip()

    # Extracting the rating
    rating = soup.find("div", class_="mm-recipes-review-bar__rating")
    if not rating:
        rating = "No rating found"
    else:
        rating = rating.get_text().strip()

    # Extracting the time to prepare
    total_time_item = soup.find("div", class_="mm-recipes-details__label", string="Total Time:")
    if total_time_item:
        total_time = total_time_item.find_next_sibling("div", class_="mm-recipes-details__value").get_text().strip()
    else:
        total_time = "No total time found"

    # Extracting the servings
    servings_item = soup.find("div", class_="mm-recipes-details__label", string="Servings:")
    if servings_item:
        servings = servings_item.find_next_sibling("div", class_="mm-recipes-details__value").get_text().strip()
    else:
        servings = "No servings found"

    # Extracting the description
    description = soup.find("meta", {"property": "og:description"})["content"]
    if not description:
        description = "No description found"
    else:
        description = description.strip()
        
    # Extracting the ingredients
    ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
    ingredients = [ingredient.get_text().strip() for ingredient in ingredients_section]

    # Extracting the instructions
    instructions_section = soup.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
    instructions = [instruction.get_text().strip() for instruction in instructions_section]

    # Extracting the nutrition information
    nutrition_section = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")
    nutrition_facts = [fact.parent.get_text().strip().replace('\n', ' ') for fact in nutrition_section]

    # Extracting image
    image = soup.find("meta", {"property": "og:image"})
    if image:
        image_url = image["content"].strip()
    else:
        image_url = "No image found"
    
    # Extracting the full text of the recipe
    full_text = f"{title}. {rating}. {total_time}. {servings}. {description}. Ingredients: {', '.join(ingredients)}. Instructions: {' '.join(instructions)}. Nutrition Facts: {' '.join(nutrition_facts)}. Image URL: {image_url}"

    # Create a dictionary to hold all the details
    recipe_details = {
        "title": title,
        "rating": rating,
        "total_time": total_time,
        "servings": servings,
        "description": description,
        "ingredients": ingredients,
        "instructions": instructions,
        "nutrition_facts": nutrition_facts,
        "image_url": image_url,
        "full_text": full_text
    }
    
    return recipe_details

In [8]:
# Call the function to get details
get_details(soup)

{'title': 'Rotisserie Chicken',
 'rating': '4.7',
 'total_time': '1 hr 30 mins',
 'servings': '6',
 'description': "Rotisserie chicken that's easy to cook on a gas grill and turns out moist and juicy with crispy skin. This is a simple recipe that our family loves.",
 'ingredients': ['1 (3 pound) whole chicken',
  '1 pinch salt',
  '¼ cup butter, melted',
  '1 tablespoon salt',
  '1 tablespoon ground paprika',
  '¼ tablespoon ground black pepper'],
 'instructions': ["Intimidated by the idea of making a rotisserie chicken at home? We're here to help. Get your grill and rotisserie attachment ready — you'll want to try this recipe ASAP.",
  "Here's what you'll need to make rotisserie chicken at home:",
  "· Whole Chicken: This recipe is meant for a whole 3-pound chicken. If your chicken is larger or smaller, you'll have to adjust the cooking time.· Butter: Butter keeps the chicken moist and juicy, while giving the seasonings something to stick to.· Seasonings: The rotisserie chicken is sim

## Parte 3: Obtener enlaces relacionados
* Encontrar links a otras recetas para completar el corpus



In [9]:
import re
def get_recipe_urls(links):
    recipe_urls = []
    for link in links:
        href = link['href']
        # Filtro específico para el patrón de AllRecipes
        if re.search(r'/recipe/\d+/[\w-]+/?$', href):
            recipe_urls.append(href)
    return recipe_urls

In [10]:
# Find all the links to other recipes
recipe_links = soup.find_all("a", href=True)

# Filter and print only the links that are likely to be recipes
recipe_urls = get_recipe_urls(recipe_links)

# Print the recipe URLs
print("Linked Recipes:")
for url in recipe_urls:
    print(url)

Linked Recipes:
https://www.allrecipes.com/recipe/238575/cilantro-lime-grilled-chicken/
https://www.allrecipes.com/recipe/275062/buttermilk-barbecue-chicken/
https://www.allrecipes.com/recipe/274724/grilled-spatchcocked-chicken/
https://www.allrecipes.com/recipe/14531/beer-butt-chicken/
https://www.allrecipes.com/recipe/221093/good-frickin-paprika-chicken/
https://www.allrecipes.com/recipe/264278/miso-honey-chicken/
https://www.allrecipes.com/recipe/258659/rosemary-buttermilk-chicken/
https://www.allrecipes.com/recipe/222936/smoked-beer-butt-chicken/
https://www.allrecipes.com/recipe/228070/the-best-beer-can-chicken-ever/
https://www.allrecipes.com/recipe/214619/bbq-beer-can-chicken/
https://www.allrecipes.com/recipe/19944/drunk-chicken/
https://www.allrecipes.com/recipe/275044/grilled-chicken-under-a-brick/
https://www.allrecipes.com/recipe/281255/smoked-whole-chicken/
https://www.allrecipes.com/recipe/34957/easy-barbeque-chicken/
https://www.allrecipes.com/recipe/8998/darn-good-chick

In [16]:
import requests

In [17]:
def extract_recipe_links(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, "html.parser")
        recipe_links = soup.find_all("a", href=True)
        return get_recipe_urls(recipe_links)
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")

In [18]:
category_urls = [
    "https://www.allrecipes.com/recipes/17562/dinner/",
    "https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/",
    "https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/",
    "https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/",
    "https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/",
    "https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/",
    "https://www.allrecipes.com/recipes/94/soups-stews-and-chili/",
    "https://www.allrecipes.com/recipes/16099/everyday-cooking/comfort-food/",
    "https://www.allrecipes.com/recipes/80/main-dish/",
    "https://www.allrecipes.com/recipes/22992/everyday-cooking/sheet-pan-dinners/",
    "https://www.allrecipes.com/recipes-a-z-6735880/",
    "https://www.allrecipes.com/recipes/78/breakfast-and-brunch/",
    "https://www.allrecipes.com/recipes/17561/lunch/",
    "https://www.allrecipes.com/recipes/84/healthy-recipes/",
    "https://www.allrecipes.com/recipes/76/appetizers-and-snacks/",
    "https://www.allrecipes.com/recipes/96/salad/",
    "https://www.allrecipes.com/recipes/81/side-dish/",
    "https://www.allrecipes.com/recipes/16369/soups-stews-and-chili/soup/",
    "https://www.allrecipes.com/recipes/156/bread/",
    "https://www.allrecipes.com/recipes/77/drinks/",
    "https://www.allrecipes.com/recipes/79/desserts/",
    "https://www.allrecipes.com/recipes/201/meat-and-poultry/chicken/",
    "https://www.allrecipes.com/recipes/200/meat-and-poultry/beef/",
    "https://www.allrecipes.com/recipes/205/meat-and-poultry/pork/",
    "https://www.allrecipes.com/recipes/93/seafood/",
    "https://www.allrecipes.com/recipes/95/pasta-and-noodles/",
    "https://www.allrecipes.com/recipes/1058/fruits-and-vegetables/fruits/",
    "https://www.allrecipes.com/recipes/1059/fruits-and-vegetables/vegetables/",
    "https://www.allrecipes.com/recipes/728/world-cuisine/latin-american/mexican/",
    "https://www.allrecipes.com/recipes/723/world-cuisine/european/italian/",
    "https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/",
    "https://www.allrecipes.com/recipes/233/world-cuisine/asian/indian/",
    "https://www.allrecipes.com/recipes/722/world-cuisine/european/german/",
    "https://www.allrecipes.com/recipes/731/world-cuisine/european/greek/",
    "https://www.allrecipes.com/recipes/696/world-cuisine/asian/filipino/",
    "https://www.allrecipes.com/recipes/699/world-cuisine/asian/japanese/"
]

In [19]:
# Processing category URLs to extract recipe links
all_recipe_urls = []
for url in category_urls:
    all_recipe_urls.extend(extract_recipe_links(url))

all_recipe_urls = list(set(all_recipe_urls))  # Remove duplicates
all_recipe_urls

['https://www.allrecipes.com/recipe/140422/onigiri-japanese-rice-balls/',
 'https://www.allrecipes.com/recipe/23893/kraut-bierocks/',
 'https://www.allrecipes.com/recipe/14759/pork-dumplings/',
 'https://www.allrecipes.com/recipe/139425/filipino-avocado-milkshake/',
 'https://www.allrecipes.com/recipe/100008/homemade-pickled-ginger-gari/',
 'https://www.allrecipes.com/recipe/8472782/frito-chicken-casserole/',
 'https://www.allrecipes.com/recipe/241502/esers-balsamic-salad-dressing/',
 'https://www.allrecipes.com/recipe/239047/one-pan-orecchiette-pasta/',
 'https://www.allrecipes.com/recipe/166667/halloumi-cheese-fingers/',
 'https://www.allrecipes.com/recipe/142881/german-schwenkbraten/',
 'https://www.allrecipes.com/recipe/260193/instant-pot-salsa-chicken/',
 'https://www.allrecipes.com/recipe/231287/sausage-potato-and-kale-soup/',
 'https://www.allrecipes.com/recipe/274884/falafel-with-canned-chickpeas/',
 'https://www.allrecipes.com/recipe/255401/chicken-afritada-filipino-stew/',
 '

In [20]:
# all_recipes
recipes_data = []
def fetch_recipe_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Call the function to get details from the fetched content
        recipe_details = get_details(soup)
        recipes_data.append(recipe_details)

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")

# Loop through the list of recipe URLs and fetch their contents
print("Recopilando recetas...")
for url in all_recipe_urls[:100]:
    fetch_recipe_content(url)

Recopilando recetas...


In [22]:
# Create a DataFrame from the recipes data
import pandas as pd
recipes_df = pd.DataFrame(recipes_data)
recipes_df.head()

,title,rating,total_time,servings,description,ingredients,instructions,nutrition_facts,image_url,full_text
0,Onigiri (Japanese Rice Balls),4.4,1 hr,4,This onigiri recipe is fun to make for Japanes...,"[4 cups uncooked short-grain white rice, 5 ½ c...",[Wash rice in a mesh strainer until water runs...,"[Total Fat 3g, Saturated Fat 1g, Sodium 160mg,...",https://www.allrecipes.com/thmb/Xed7WCnxYS6lPa...,Onigiri (Japanese Rice Balls). 4.4. 1 hr. 4. T...
1,Kraut Bierocks,4.6,1 hr 45 mins,10,These meat-mixture filled rolls make a great m...,"[1 ½ (.25 ounce) packages active dry yeast, ½ ...",[To Make Sweet Dough: In a medium bowl combine...,"[Total Fat 34g, Saturated Fat 13g, Cholesterol...",https://www.allrecipes.com/thmb/Z2_ZSaelT_MOC4...,Kraut Bierocks. 4.6. 1 hr 45 mins. 10. These m...
2,Pork Dumplings,4.7,1 hr,10,These amazingly easy steamed pork dumplings ha...,"[100 (3.5 inch square) wonton wrappers, 1 ¾ p...",[If you're looking for the best steamed pork d...,"[Total Fat 29g, Saturated Fat 9g, Cholesterol ...",https://www.allrecipes.com/thmb/tCt2q6Sly6Spyu...,Pork Dumplings. 4.7. 1 hr. 10. These amazingly...
3,Filipino Avocado Milkshake,4.7,5 mins,2,"This avocado milkshake made with milk, sugar, ...","[1 medium avocado - peeled, pitted, and cubed,...","[Combine avocado, milk, ice cream, ice, sugar,...","[Total Fat 19g, Saturated Fat 5g, Cholesterol ...",https://www.allrecipes.com/thmb/d-YvB22kgzjN3T...,Filipino Avocado Milkshake. 4.7. 5 mins. 2. Th...
4,Homemade Pickled Ginger (Gari),4.7,45 mins,32,"Pickled ginger, or gari, is served as a palate...","[8 ounces fresh young ginger root, peeled, 1 ½...","[Gather all ingredients., Cut ginger into chun...","[Total Fat 0g, Sodium 83mg, Total Carbohydrate...",https://www.allrecipes.com/thmb/AiFUs73Pna0irz...,Homemade Pickled Ginger (Gari). 4.7. 45 mins. ...


In [21]:
## Segundo Método para buscar Recetas
# Buscar todos los enlaces <a href="..."> en el HTML

#recipe_links = soup.find_all("a", href=True)

# Filtrar solo los que empiezan con el patrón deseado
#recipe_urls = []
#for link in recipe_links:
#    href = link['href']
#    if href.startswith("https://www.allrecipes.com/recipe"):
#        recipe_urls.append(href)

# Eliminar duplicados (opcional pero recomendado)
#recipe_urls = list(set(recipe_urls))

# Mostrar los enlaces filtrados
#print("🔗 Enlaces a recetas:")
#for url in recipe_urls:
#    print(url)


# Parte 4: Hacer RAG con las recetas obtenidas
* Una vez que se ha construido el corpus, implementar y desplegar RAG para realizar búsquedas en el corpus


In [23]:
# Create embeddings for the recipes data
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Generando embeddings...")
embeddings = model.encode(recipes_df['full_text'].tolist(), convert_to_numpy=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generando embeddings...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
# Create index FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)  # Add embeddings to the index
# Adding embeddings to DataFrame
recipes_df['embeddings'] = embeddings.tolist()  # Convert numpy array to list for Data

In [26]:
def seeker(query, k=5):
    """
    Function to search for recipes based on a query.
    Returns the top k recipes based on cosine similarity.
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    
    context_recipes = []
    recipe_summaries = []
    for i in range(k):
        recipe = recipes_df.iloc[indices[0][i]]
        context_recipes.append({
            "title": recipe['title'],
            "rating": recipe['rating'],
            "total_time": recipe['total_time'],
            "servings": recipe['servings'],
            "description": recipe['description'],
            "ingredients": recipe['ingredients'],
            "instructions": recipe['instructions'],
            "nutrition_facts": recipe['nutrition_facts'],
            "image_url": recipe['image_url'],
            "distance": distances[0][i]
        })
        
        # Create a summary for the recipe
        summary = f"{recipe['title']} - {recipe['rating']} stars, {recipe['total_time']} to prepare, serves {recipe['servings']}. {recipe['description']}. Ingredients: {', '.join(recipe['ingredients'])}. Instructions: {' '.join(recipe['instructions'])}. Nutrition Facts: {' '.join(recipe['nutrition_facts'])}. Image URL: {recipe['image_url']}"
        recipe_summaries.append(summary)
    
    # Create prompt for ChatGPT
    prompt = f"""
    Basándote únicamente en las siguientes recetas, responde la consulta del usuario: "{query}"
    
    Recetas relevantes:
    {chr(10).join(recipe_summaries)}
    
    Proporciona una respuesta útil y específica basada en estas recetas. Si es apropiado, recomienda una receta específica y explica por qué.
    """
    
    # Generate response using OpenAI
    response = client.responses.create(
        model="gpt-4.1",
        input=prompt
    )

    return response.output_text

In [29]:
query = "¿Dame la receta de la Sopa de pollo?"
response = seeker(query)
print("Respuesta de ChatGPT:")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Respuesta de ChatGPT:
¡Claro! Basándome en las recetas proporcionadas, la mejor receta que corresponde a "Sopa de pollo" es el **Caldo de Pollo**, que es una sopa clásica de pollo mexicana, llena de sabor y vegetales. Aquí tienes la receta detallada:

---

## Receta de Sopa de Pollo (Caldo de Pollo)

### Ingredientes
- 5 libras de muslos o piernas de pollo
- 2 galones de agua
- 2 cucharadas de ajo picado
- 2 cucharadas de sal
- 1 cucharada de ajo en polvo
- 1 cubo de consomé de pollo
- 4 zanahorias grandes, peladas y cortadas en trozos grandes
- 4 papas grandes, peladas y cortadas en trozos grandes
- 4 calabacitas, cortadas en trozos grandes
- 1 chayote, cortado en trozos grandes
- 1 cebolla blanca grande, cortada en trozos grandes
- ½ manojo de cilantro fresco, picado

### Instrucciones
1. **Prepara los ingredientes:** Junta todos los ingredientes y corta las verduras en trozos grandes.
2. **Cocina el pollo:** Coloca los muslos o piernas de pollo en una olla grande. Vierte el agua sob